# 용어들

## 시소러스?thesaurus!  
-단어의 의미를 사전이 아니라 일종의 유의어 사전처럼 구성하는 것.  
-예를 들어, 자동차 = 자동, 차, 기계  
-이것을 그래프, 트리 형식으로 표현하면 단어의 관계를 알수 있고, 컴퓨터에게도 단어를 알수있다(?)고 할수 있음.

## 시소러스의 문제점?  
-시대의 변화에 대응하기 어렵다. 즉, 다양한 단어들이 나타나기 때문에 그런 단어들의 관계를 다시 표현해야함.  
-사람이 수작업이 들어가야하기 때문에 휴먼 리소스가 크다.  
-유의어로 구성되어 있기때문에 그 단어들 사이의 비묘한 차이들까지는 표현하기 어렵다.

## corpus(말뭉치)  
-쉽게 말해서 대량의 텍스트 데이터.  
-사람들은 통계적 기법을 통해서 코퍼스에서 단어들 사이의 관게를 자동으로 표현하고 싶어함.

# python으로 코퍼스 전처리하기

In [18]:

text = 'You say goodbye and I say hello.'

In [7]:
text = text.lower()
text = text.replace('.','..')

In [8]:
text

'you say goodbye and i say hello..'

In [10]:
words = text.split()
words

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello..']

In [15]:
word_to_id = {}
id_to_word = {}

for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word]=new_id
        id_to_word[new_id]=word

word_to_id
id_to_word

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello..'}

In [17]:
def preprocess(text):
    text = text.lower()
    text = text.replace('.',' .')
    
    word_to_id = {}
    id_to_word = {}

    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word]=new_id
            id_to_word[new_id]=word
    corpus = np.array([word_to_id[w] for w in words])
    return corpus, word_to_id, id_to_word
    
    
    

## 단어의 분산표현  
-색을 표현하는 단어가 여러가지임. 하지만 그 색들도 RGB의 값 조합으로 나타낼수 있듯이 단어들도 그렇게 표현해보고자 하는것.  
  
분산가설  
-단어의 의미는 주변단어에의해 형성된다.맥락!  

## 동시발생 행렬  
-가장 간단한 방법으로 특정 단어의 주변에 어떤 단어들이 얼마나 나왔는지를 통계내보는것.  
-예를들어, you say goodbye and i say hello.라는 문장이 있을떄.  
say = [1,0, 1, 0, 1, 1, 0] 으로 나타낼수 있음.

In [19]:
import numpy as np

In [23]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size),dtype=np.int32)
    
    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size+1):
            left_idx = idx - i
            right_idx = idx + 1
            
            if left_idx >= 0 :
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] +=1
                
            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] +=1
    return co_matrix
        
        

## 백터간 유사도 - 코사인 유사도  
-0으로 나눠지는 경우에 대비해서 입실론, 아주작은값을 더해준다

In [25]:
def cos_similarity(x,y, eps=1e-8):
    nx = x/(np.sqrt(np.sum(x**2))+eps)
    ny = y/(np.sqrt(np.sum(y**2))+eps)
    return np.dot(nx, ny)

## 상호정보량  
-발생확률만으로는 부족함.  
-예를들어 the같은 경우에는 관련성보다는 많이 출현하는것때문에 형렬에 자주 나타나게 됨.  
-이것을 해결하기 위한 방법이 PMI!  
logp(x,y)/p(x)p(y)  
-근데 위와 마찬가지로 0값을 가질때 무한대의 값이 나오게 되므로 ppmi를 사용.  
PPMI(x,y) = max(0, pmi(x,y))

## 차원 감소  
-데이터의 차원을 중요한 정보는 살리면서 줄이는것. 예시로 SVD, PCA등등...